In [19]:
import pandas as pd
import numpy as np

In [20]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset

import pytorch_lightning as pl
from pytorch_lightning.metrics import F1
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.loggers import TensorBoardLogger

In [21]:
from sklearn.metrics import (
    f1_score,
    accuracy_score,
    classification_report,
    confusion_matrix,
)

In [22]:
btc = pd.read_csv("../data/0_raw/BTC_USD_2013-10-01_2021-04-21-CoinDesk.csv")

In [23]:
btc.columns

Index(['Currency', 'Date', 'Closing Price (USD)', '24h Open (USD)',
       '24h High (USD)', '24h Low (USD)'],
      dtype='object')

In [24]:
btc.Date = btc.Date.apply(pd.Timestamp)

In [25]:
btc = btc.sort_values("Date", ascending=True)

In [26]:
btc

,Currency,Date,Closing Price (USD),24h Open (USD),24h High (USD),24h Low (USD)
0,BTC,2013-10-01,123.654990,124.304660,124.751660,122.563490
1,BTC,2013-10-02,125.455000,123.654990,125.758500,123.633830
2,BTC,2013-10-03,108.584830,125.455000,125.665660,83.328330
3,BTC,2013-10-04,118.674660,108.584830,118.675000,107.058160
4,BTC,2013-10-05,121.338660,118.674660,121.936330,118.005660
...,...,...,...,...,...,...
2754,BTC,2021-04-17,61965.782598,63225.093917,63520.325374,60033.534667
2755,BTC,2021-04-18,60574.444728,61444.232503,62534.028498,59802.889267
2756,BTC,2021-04-19,56850.830166,60191.525406,60531.988848,52148.983544
2757,BTC,2021-04-20,56224.101588,56335.389141,57609.368118,54449.245330


In [27]:
btc = btc[(btc.Date >= pd.Timestamp("2017"))]

In [28]:
len(btc)

1571

In [29]:
int(len(btc)*7/10)

1099

In [30]:
btc = btc[['24h Open (USD)', '24h High (USD)', '24h Low (USD)', 'Closing Price (USD)']]

In [31]:
btc.columns = ["open", "high", "low", "close"]

class TimeSeriesDataset(Dataset):
    def __init__(self, x: np.ndarray, seq_len = WINDOW_SIZE):
        self.x = torch.tensor(x).float()
        self.seq_len = seq_len
        
    def __len__(self):
        #return len(self.x) - ( self.seq_len -1 ) #sliding window count
        return len(self.x) - ( self.seq_len)
    
    def __getitem__(self, index):
        #return (self.x[index:index+self.seq_len], self.x[index+self.seq_len]) # regression
        
        window = self.x[index:index+self.seq_len]
        price_change = self.x[index+self.seq_len] - self.x[index+self.seq_len-1]
        price_change = 0 if price_change == 0 else 1 if price_change>0 else 2 #2 düşüş
        return (window, price_change)

In [32]:
time_series = btc.close.round(2).to_numpy()

In [33]:
btc.close

1188      964.325000
1189     1009.973750
1190     1028.333750
1191     1047.099990
1192     1140.385000
            ...     
2754    61965.782598
2755    60574.444728
2756    56850.830166
2757    56224.101588
2758    56608.769748
Name: close, Length: 1571, dtype: float64

In [34]:
N_CLASSES = 2 ###
N_CURRENCIES = 1
INPUT_FEATURE_SIZE = 1
WINDOW_SIZE = 50
TRAIN_PERCENTAGE, VAL_PERCENTAGE, TEST_PERCENTAGE = 0.80, 0.10, 0.10

In [35]:
#train dataset ile val dataset ayırabilirim okunması daha kolay olsun diye aslında

In [36]:
class TimeSeriesDataset(Dataset):
    def __init__(self, 
                 x: np.ndarray, 
                 data_use_type,
                 train_percentage = TRAIN_PERCENTAGE,
                 val_percentage = VAL_PERCENTAGE,
                 test_percentage = TEST_PERCENTAGE,
                 seq_len = WINDOW_SIZE, 
                 ):
        
        self.x = torch.tensor(x).float()
        self.seq_len = seq_len
        
        self.data_use_type = data_use_type
        
        self.train_size = int(len(self.x) * train_percentage)
        self.val_size = int(len(self.x) * val_percentage)
        self.test_size = int(len(self.x) * test_percentage)
        
        self.train_mean = self.x[:self.train_size].mean()
        self.train_std = self.x[:self.train_size].std()
        
        self.train_min = self.x[:self.train_size].min()
        self.train_max = self.x[:self.train_size].max()
        
    def __len__(self):
        
        if self.data_use_type == "train":
            return self.train_size - ( self.seq_len)
        
        if self.data_use_type == "val":
            return self.val_size
        
        else:
            return self.test_size
        
    def __getitem__(self, index):
        
        if self.data_use_type =="val":
            index = self.train_size + index - self.seq_len
            
        elif self.data_use_type =="test":
            index = self.train_size + self.val_size + index - self.seq_len
        
        window = self.x[index:index+self.seq_len]
        window = (window -self.train_mean) / self.train_std 
        
        price_change = self.x[index+self.seq_len] - self.x[index+self.seq_len-1]
        price_change =  2 if price_change == 0 else 1 if price_change >0 else 0 #2 değişmemesi, 1 artış, 0 düşüş
        #burada 2 hiç gelmiyor N_CLASSES = 2 dediğinde run edince patlardı
        return (window, price_change)

In [37]:
train_dataset = TimeSeriesDataset(time_series, "train")
val_dataset = TimeSeriesDataset(time_series, "val")
test_dataset = TimeSeriesDataset(time_series, "test")

In [38]:
test_dataset.__len__()

157

In [39]:
#calculate loss' weights
train_labels = [train_dataset[i][1] for i in range (train_dataset.__len__())]
samples_size = pd.DataFrame({"label": train_labels}).groupby("label").size().to_numpy()
loss_weights = (1 / samples_size) * sum(samples_size)/2
loss_weights

array([1.09836066, 0.91780822])

In [40]:
class LSTM_based_classification_model(pl.LightningModule):
    def __init__(self,
                 train_dataset = train_dataset,
                 val_dataset = val_dataset,
                 test_dataset = test_dataset,
                 weights = loss_weights,
                 num_classes = N_CLASSES,
                 window_size = WINDOW_SIZE,
                 input_size = INPUT_FEATURE_SIZE,
                 batch_size=8,
                 lstm_hidden_size = 256,
                 lstm_stack_size = 1,
                 lstm_dropout = 0.2,
                 bidirectional = False,
                 ):
        
        super().__init__()
        self.num_classes = num_classes
        self.window_size = window_size
        self.input_size = input_size
        self.batch_size = batch_size
        
        self.lstm_hidden_size = lstm_hidden_size
        self.lstm_stack_size = lstm_stack_size
        self.lstm_dropout = lstm_dropout
        self.bidirectional = bidirectional 
        
#         self.stack_lstm = nn.LSTM(input_size = self.input_size, 
#                 hidden_size = self.lstm_hidden_size, 
#                 num_layers= self.lstm_stack_size,
#                 dropout = self.lstm_dropout, # sadece stack arasına koyuyor
#                 bidirectional = self.bidirectional, 
#                 batch_first=True,) 
        
        self.lstm_1 = nn.LSTM(input_size = self.input_size, num_layers=1, batch_first=True, hidden_size = 256)
        self.batch_norm1 = nn.BatchNorm2d(num_features=256)
        
        self.lstm_2 = nn.LSTM(input_size = 256, num_layers=1, batch_first=True, hidden_size = 128)
        self.batch_norm2 = nn.BatchNorm2d(num_features=128)
        
        self.lstm_3 = nn.LSTM(input_size = 128, num_layers=1, batch_first=True, hidden_size = 64)
        self.batch_norm3 = nn.BatchNorm2d(num_features=64)
        
        self.dropout = nn.Dropout(0.5)
        
        #         self.linear1 = nn.Linear(self.lstm_hidden_size, 128)
        
        self.linear1 = nn.Linear(64, 128)
        
        self.activation = nn.ReLU()
        

        self.output_layer = nn.Linear(128, self.num_classes)
        
        #self.cross_entropy_loss = nn.CrossEntropyLoss(weight= torch.tensor(weights).float()) # loss weight
        self.cross_entropy_loss = nn.CrossEntropyLoss()
        
        self.f1_score = pl.metrics.F1(num_classes=self.num_classes, average="macro")
        self.accuracy_score = pl.metrics.Accuracy()
        
        self.train_dl = DataLoader(train_dataset, batch_size=self.batch_size, )
        self.val_dl = DataLoader(val_dataset, batch_size=self.batch_size)
        self.test_dl = DataLoader(test_dataset, batch_size=self.batch_size)
        
    def forward(self, x):
        batch_size = x.size()[0]
        
        x = x.view(batch_size, self.window_size, self.input_size) #(batch, window_len, feature_size)
        x, _  = self.lstm_1(x)
        
        x = x.reshape(256, batch_size, self.window_size) #(feature_size, batch, window_len)
        x = self.batch_norm1(x.unsqueeze(0))
        
        x = x.view(batch_size, self.window_size, 256)
        x, _  = self.lstm_2(x)
        
        x = x.reshape(128, batch_size, self.window_size) #(feature_size, batch, window_len)
        x = self.batch_norm2(x.unsqueeze(0))
        #         x, _ = self.lstm2(x)
        
        x = x.view(batch_size, self.window_size, 128)
        x, _  = self.lstm_3(x)
        
        x = x.reshape(64, batch_size, self.window_size) #(feature_size, batch, window_len)
        x = self.batch_norm3(x.unsqueeze(0))
        
        x = x.view(batch_size, self.window_size, 64)
        x = x[:, -1, :] # equivalent to return sequence = False on keras :)
        
#         x = self.dropout(x)
        
        x = self.linear1(x)
        x = self.activation(x)
        
#         x = self.linear2(x)
#         x = self.activation(x)
        x = self.dropout(x)
            
        output = self.output_layer(x)
        
        #output = F.log_softmax(output, dim = 1)
        #output = F.softmax(output)
        
        return output
    
    def training_step(self, batch, batch_nb):
        x, y = batch
        output = self.forward(x)
        #loss = F.nll_loss(output, y)
        loss = self.cross_entropy_loss(output, y)
        
        self.log('train_loss', loss, on_epoch=True, prog_bar=True)
        
        acc = self.accuracy_score(torch.max(output, dim=1)[1], y)
        self.log('train_acc', acc, on_epoch=True, prog_bar=True)
        
        f1 = self.f1_score(torch.max(output, dim=1)[1], y)
        self.log('train_f1', f1, on_epoch=True, prog_bar=True)
        
        return loss
    
    def validation_step(self, batch, batch_nb):
        x, y = batch
        output = self(x)
        #loss = F.nll_loss(output, y)
        loss = self.cross_entropy_loss(output, y)
        self.log('val_loss', loss, on_epoch=True, reduce_fx=torch.mean, prog_bar=True)
        
        #print(torch.max(output, dim=1)[1])
        acc = self.accuracy_score(torch.max(output, dim=1)[1], y)
        self.log('val_acc', acc, on_epoch=True, reduce_fx=torch.mean, prog_bar=True)
        
        f1 = self.f1_score(torch.max(output, dim=1)[1], y)
        self.log('val_f1', f1, on_epoch=True, reduce_fx=torch.mean, prog_bar=True)
        
    def test_step(self, batch, batch_nb):
        x, y = batch
        output = self(x)
        
        #print(y, torch.max(output, dim=1)[1])
        #print(output)
        print(F.softmax(output)) # mantıken fark etmiyor
        print(y, torch.max(output, dim=1)[1])
        
        #loss = F.nll_loss(output, y)
        loss = self.cross_entropy_loss(output, y)
        self.log('test_loss', loss, on_epoch=True, reduce_fx=torch.mean)
        
        acc = self.accuracy_score(torch.max(output, dim=1)[1], y)
        self.log('test_acc', acc, on_epoch=True, reduce_fx=torch.mean)
        
        f1 = self.f1_score(torch.max(output, dim=1)[1], y)
        self.log('test_f1', f1, on_epoch=True, reduce_fx=torch.mean)
        
    def configure_optimizers(self):
        
        optimizer = torch.optim.AdamW(model.parameters(), lr= 1e-3)#AdamW does weight decay
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

        return [optimizer], [{"scheduler": scheduler}]
    
    def train_dataloader(self):
        return self.train_dl

    def val_dataloader(self):
        return self.val_dl

    def test_dataloader(self):
        return self.test_dl

In [41]:
!rm -rf ../output/models/lstm_v1/version_*

In [42]:
#logger = WandbLogger(name='lstm.v4',project='pytorchlightning')
logger = TensorBoardLogger("../output/models/lstm_model_logs", name="lstm_v1")

In [43]:
model = LSTM_based_classification_model(batch_size=8)
trainer = pl.Trainer(gpus=-1, 
                     logger = logger)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [45]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name               | Type             | Params
---------------------------------------------------------
0  | lstm_1             | LSTM             | 265 K 
1  | batch_norm1        | BatchNorm2d      | 512   
2  | lstm_2             | LSTM             | 197 K 
3  | batch_norm2        | BatchNorm2d      | 256   
4  | lstm_3             | LSTM             | 49.7 K
5  | batch_norm3        | BatchNorm2d      | 128   
6  | dropout            | Dropout          | 0     
7  | linear1            | Linear           | 8.3 K 
8  | activation         | ReLU             | 0     
9  | output_layer       | Linear           | 258   
10 | cross_entropy_loss | CrossEntropyLoss | 0     
11 | f1_score           | F1               | 0     
12 | accuracy_score     | Accuracy         | 0     
---------------------------------------------------------
521 K     Trainable params
0         Non-trainable params
521 K     Total params
2.088     Total estimated model 

/home/aysenurk/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/home/aysenurk/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1

In [46]:
trainer.test()

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/aysenurk/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


<ipython-input-40-1fa05d6bbe1e>:142: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print(F.softmax(output)) # mantıken fark etmiyor


tensor([[0.4476, 0.5524],
        [0.4478, 0.5522],
        [0.4476, 0.5524],
        [0.4486, 0.5514],
        [0.4510, 0.5490],
        [0.4483, 0.5517],
        [0.4464, 0.5536],
        [0.4481, 0.5519]], device='cuda:0')
tensor([0, 0, 1, 1, 1, 1, 1, 0], device='cuda:0') tensor([1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
tensor([[0.4453, 0.5547],
        [0.4450, 0.5550],
        [0.4446, 0.5554],
        [0.4456, 0.5544],
        [0.4483, 0.5517],
        [0.4457, 0.5543],
        [0.4450, 0.5550],
        [0.4466, 0.5534]], device='cuda:0')
tensor([1, 0, 1, 0, 0, 0, 1, 1], device='cuda:0') tensor([1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
tensor([[0.4444, 0.5556],
        [0.4444, 0.5556],
        [0.4444, 0.5556],
        [0.4453, 0.5547],
        [0.4480, 0.5520],
        [0.4454, 0.5546],
        [0.4446, 0.5554],
        [0.4462, 0.5538]], device='cuda:0')
tensor([1, 0, 1, 1, 0, 1, 1, 0], device='cuda:0') tensor([1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
tensor([[0.4441, 0.555

[{'test_loss': 0.6828314661979675,
  'test_acc': 0.5796178579330444,
  'test_f1': 0.35844093561172485}]

In [ ]:
test = TimeSeriesDataset(time_series, "test")
labels = [test[i][1] for i in range(test.__len__())]

In [ ]:
pd.DataFrame({"label": labels}).groupby("label").size()

In [ ]:
#dropout, batch normalization 

In [ ]:
183 / (183+131)